In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the btuB gene
model0['lb_list']['EX_cbl1_e ']=0
model0['lb_list']['EX_cbl1_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                                    if i == 0:
                                        # gene upregulation for Fe2+ intake
                                        if 'EX_fe2_e' in rea:
                                            public_reaction_ub_list[i][rea][0, m] = public_reaction_ub_list[i][rea][0, m] * 1.2
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([12,  9, 17, 17,  2,  2,  2, 10,  8, 17,  1,  8,  6, 10, 17,  1,  1,
        4, 10,  9,  2, 16,  4,  6,  1,  5, 18, 17,  6, 11]), 1: array([14, 16,  4, 17, 20, 16, 17,  9, 18,  4, 18, 14, 11, 10, 15, 17,  3,
       18,  6, 20, 11,  4,  3, 10,  8,  3, 14, 12,  6,  1])}
1
strain_number:  0 9.6
strain_various:  0 6.907001278895689
strain_number:  1 11.3
strain_various:  1 5.808901215663194
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([12,  9, 17, 17,  2,  2,  2, 10,  8, 17,  1,  8,  6, 10, 17,  1,  1,
        4, 10,  9,  2, 16,  4,  6,  1,  5, 18, 17,  6, 11]), 1: array([14, 16,  4, 17, 20, 16, 17,  9, 18,  4, 18, 14, 11, 10, 15, 17,  3,
       18,  6, 20, 11,  4,  3, 10,  8,  3, 14, 12,  6,  1])}
_2:  {0: [0.6784862515716307, 0.2857391744693639], 1: [0.6784862515716307, 0.2857391744693639], 2: [0.6784862515716307, 0.2857391744693639], 3: [0.6784862515716307, 0.2857391744693639], 4: [0.6784862515716307, 0.2857391744693639], 5: [0.6784862515716307, 0.285

glc__D_e_1:  -10.326339076416016
glc__D_e_1:  -0.9732073982806169
glc__D_e_1:  -6.0161697228218225
glc__D_e_1:  -10.66539490223342
glc__D_e_1:  -8.161350457736257
glc__D_e_1:  -20.058939691189163
glc__D_e_1:  -0.015328238053344378
glc__D_e_1:  -1.7568466345876879
glc__D_e_1:  -15.767866839840151
glc__D_e_1:  -18.10733422878889
glc__D_e_1:  -2.4554936653753554
glc__D_e_1:  -23.685484580607742
glc__D_e_1:  -13.582524746667726
glc__D_e_1:  -2.512318372540287
glc__D_e_1:  -0.74684631424077
glc__D_e_1:  -12.092029372007904
glc__D_e_1:  -24.003809658598506
glc__D_e_1:  -14.112686092377652
glc__D_e_1:  -29.01135040217
strain_number:  0 38.266666666666666
strain_various:  0 15.800703219231169
strain_number:  1 1.4333333333333333
strain_various:  1 1.5849991237291652
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 791.6644898574077, 397.68279527490046, 386.4915687382793, 420.59968211612943, 434.4683857808859, 370.33773463378736, 442.70701009516796, 477.57966054808156, 490.6352957696722, 372.62902270

glc__D_e_1:  -0.049587727791070346
glc__D_e_1:  -0.03752192992285802
glc__D_e_1:  -0.3089706377373267
glc__D_e_1:  -1.2673242868012482
glc__D_e_1:  -0.6508694530937948
glc__D_e_1:  -1.0912805902850076
glc__D_e_1:  -2.155351747542005
glc__D_e_1:  -0.3161503577329553
glc__D_e_1:  -3.9075245742784235
glc__D_e_1:  -4.154646884541207
glc__D_e_1:  -6.750239823820911
glc__D_e_1:  -4.8874847950779845
glc__D_e_1:  -9.837905811679327
glc__D_e_1:  -7.186646685849865
glc__D_e_1:  -10.659233947849664
glc__D_e_1:  -5.520692121609567
glc__D_e_1:  -10.157430304035145
glc__D_e_1:  -6.075326355572402
glc__D_e_1:  -1.8533430764264132
glc__D_e_1:  -3.468703492998526
glc__D_e_1:  -4.79530310308233
glc__D_e_1:  -1.8595670493127896
glc__D_e_1:  -1.2565846091723423
glc__D_e_1:  -0.9229183937129821
glc__D_e_1:  -0.08906728031389766
strain_number:  0 4.5
strain_various:  0 4.387482193696061
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 791.6644898574077, 397.68279527490

In [7]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2,  2, 13, 11,  4, 15, 17,  2,  2,  5,  4, 15, 20,  8, 18, 14, 15,
        5,  6, 12, 14,  1,  8, 18,  1,  7,  3,  4, 10,  1]), 1: array([ 1, 10, 14,  1,  6,  5, 12, 20, 15, 14,  8,  5,  6, 10, 20, 16,  3,
        6, 17,  9,  3,  7,  4,  3,  5,  9, 19,  2, 13,  3])}
1
strain_number:  0 9.9
strain_various:  0 7.208097298640374
strain_number:  1 8.866666666666667
strain_various:  1 5.754611677216419
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2,  2, 13, 11,  4, 15, 17,  2,  2,  5,  4, 15, 20,  8, 18, 14, 15,
        5,  6, 12, 14,  1,  8, 18,  1,  7,  3,  4, 10,  1]), 1: array([ 1, 10, 14,  1,  6,  5, 12, 20, 15, 14,  8,  5,  6, 10, 20, 16,  3,
        6, 17,  9,  3,  7,  4,  3,  5,  9, 19,  2, 13,  3])}
_2:  {0: [0.6459265309332798, 0.2857391744693639], 1: [0.6459265309332798, 0.2857391744693639], 2: [0.6459265309332798, 0.2857391744693639], 3: [0.6459265309332798, 0.2857391744693639], 4: [0.6459265309332798, 0.2857391744693639], 5: [0.6459265309

glc__D_e_1:  -1.6749992366942195
glc__D_e_1:  -11.052817644941875
glc__D_e_1:  -18.383620904266277
glc__D_e_1:  -20.35885846072138
glc__D_e_1:  -13.622600365777213
glc__D_e_1:  -13.686800453573987
glc__D_e_1:  -10.258728248692448
glc__D_e_1:  -13.342979512567135
glc__D_e_1:  -7.835296626926606
glc__D_e_1:  -11.243629299511712
glc__D_e_1:  -11.517025067206221
glc__D_e_1:  -17.387098260710122
glc__D_e_1:  -24.924839924062738
glc__D_e_1:  -15.773998815204347
glc__D_e_1:  -0.6898747381135024
strain_number:  0 39.333333333333336
strain_various:  0 18.928520515055816
strain_number:  1 0.06666666666666667
strain_various:  1 0.24944382578492943
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 888.5989542578164, 345.8598616295985, 362.65257659007654, 427.3235496959182, 513.0633388541283, 525.9186307749976, 536.2511567379988, 454.8808372999549, 473.2230091316634, 443.28945622441915, 415.2661198354646, 357.2583756806445, 295.678261585683, 295.0243320594571, 222.9252202880938]
slip_r:  222.9252202880938

glc__D_e_1:  -3.1306741529951623
glc__D_e_1:  -12.585284490533176
glc__D_e_1:  -12.3088033192047
glc__D_e_1:  -12.851466367050012
glc__D_e_1:  -37.024167856660135
glc__D_e_1:  -12.031756909053414
strain_number:  0 7.033333333333333
strain_various:  0 9.159633665648911
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 888.5989542578164, 345.8598616295985, 362.65257659007654, 427.3235496959182, 513.0633388541283, 525.9186307749976, 536.2511567379988, 454.8808372999549, 473.2230091316634, 443.28945622441915, 415.2661198354646, 357.2583756806445, 295.678261585683, 295.0243320594571, 222.9252202880938, 205.1847833999331, 180.9789908536638, 165.78459621920177, 163.61977946966425, 218.31009624125457, 193.69590759002523]
slip_r:  193.69590759002523
23
glc__D_e_1:  -6.293754226018169
glc__D_e_1:  -3.8093023437782065
glc__D_e_1:  -6.528599090497691
glc__D_e_1:  -4.474712224470595
glc__D_e_1:  -3.4506539538120706
glc__D_e_1:  -2.3493386896346085
glc__D_e_1:  

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([18,  3,  9, 10, 15, 17,  5,  1, 19,  6, 13,  4, 14,  1, 20, 19, 14,
        9,  8, 18, 16,  3, 11,  7, 15, 17,  7,  9, 20, 18]), 1: array([11, 17,  9,  9, 16,  1, 12,  6, 17,  3, 15,  4, 17, 15, 16,  6, 13,
       20, 10,  2,  4,  8,  4, 17,  7, 14, 15,  5,  8,  3])}
1
strain_number:  0 13.4
strain_various:  0 7.176814520848834
strain_number:  1 10.133333333333333
strain_various:  1 5.475602452901619
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([18,  3,  9, 10, 15, 17,  5,  1, 19,  6, 13,  4, 14,  1, 20, 19, 14,
        9,  8, 18, 16,  3, 11,  7, 15, 17,  7,  9, 20, 18]), 1: array([11, 17,  9,  9, 16,  1, 12,  6, 17,  3, 15,  4, 17, 15, 16,  6, 13,
       20, 10,  2,  4,  8,  4, 17,  7, 14, 15,  5,  8,  3])}
_2:  {0: [0.5207337757122594, 0.2857391744693639], 1: [0.5207337757122594, 0.2857391744693639], 2: [0.5207337757122594, 0.2857391744693639], 3: [0.5207337757122594, 0.2857391744693639], 4: [0.5207337757122594, 0.2857391744693639], 5: [0.52073377

glc__D_e_1:  -18.752229447087938
glc__D_e_1:  -24.820969713877947
glc__D_e_1:  -0.30006196864939305
glc__D_e_1:  -12.48813230596744
glc__D_e_1:  -27.629815554375476
glc__D_e_1:  -0.7354341114507754
glc__D_e_1:  -15.832654008262814
glc__D_e_1:  -29.110477904992493
glc__D_e_1:  -47.14524570481466
glc__D_e_1:  -21.179281711251477
glc__D_e_1:  -0.552921536783396
glc__D_e_1:  -26.222166529631163
glc__D_e_1:  -36.15621712327485
glc__D_e_1:  -0.3191182181634886
glc__D_e_1:  -44.08627628286875
glc__D_e_1:  -27.07798121584878
glc__D_e_1:  -13.941374046977085
glc__D_e_1:  -24.27455652376858
glc__D_e_1:  -11.770150502646832
glc__D_e_1:  -18.456948248161204
glc__D_e_1:  -15.431126329571354
strain_number:  0 37.46666666666667
strain_various:  0 14.263551061670755
strain_number:  1 1.5
strain_various:  1 1.5
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 899.1394597404658, 321.9547680389786, 416.3453377762588, 427.9996295441967, 433.7504069802418, 410.46084249484454, 367.2693817036459, 412.5315510392, 3

glc__D_e_1:  -2.4436660833296955
glc__D_e_1:  -2.9867157997712637
glc__D_e_1:  -1.4516398422705317
glc__D_e_1:  -4.0008605545555
glc__D_e_1:  -1.3878696616657682
glc__D_e_1:  -2.8789011492671293
glc__D_e_1:  -0.6556554589927313
glc__D_e_1:  -1.3220625779524646
glc__D_e_1:  -1.7135697085208488
glc__D_e_1:  -0.40019705826125884
glc__D_e_1:  -0.4577507858351436
strain_number:  0 3.8
strain_various:  0 1.8690461025168248
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 899.1394597404658, 321.9547680389786, 416.3453377762588, 427.9996295441967, 433.7504069802418, 410.46084249484454, 367.2693817036459, 412.5315510392, 364.9083852522315, 390.9539582770756, 364.67973395820405, 319.8268757141048, 361.0992699748281, 395.6695563987873, 338.9954604179019, 192.28083314334367, 215.7426300479706, 216.42045454545453]
slip_r:  216.42045454545453
20
glc__D_e_1:  -0.7456891511820947
glc__D_e_1:  -0.31870142564181747
glc__D_e_1:  -1.1493501457161572
glc__D_e_1:  -0.4

In [7]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([12,  2, 20,  8,  5, 12,  3, 12, 14, 20, 12,  4,  6, 18,  3, 14, 17,
       17, 14, 19,  2, 13, 17,  7, 10, 10,  8, 19, 10, 17]), 1: array([15,  6, 14, 18,  6,  1, 14, 19, 20,  9,  1, 18,  8, 14, 16, 11, 20,
        9,  5, 16, 12,  7,  4,  3, 11, 13, 14, 11, 14,  1])}
1
strain_number:  0 13.366666666666667
strain_various:  0 6.740342886101732
strain_number:  1 11.0
strain_various:  1 5.692099788303083
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([12,  2, 20,  8,  5, 12,  3, 12, 14, 20, 12,  4,  6, 18,  3, 14, 17,
       17, 14, 19,  2, 13, 17,  7, 10, 10,  8, 19, 10, 17]), 1: array([15,  6, 14, 18,  6,  1, 14, 19, 20,  9,  1, 18,  8, 14, 16, 11, 20,
        9,  5, 16, 12,  7,  4,  3, 11, 13, 14, 11, 14,  1])}
_2:  {0: [0.5090172841115191, 0.2857391744693639], 1: [0.5090172841115191, 0.2857391744693639], 2: [0.5090172841115191, 0.2857391744693639], 3: [0.5090172841115191, 0.2857391744693639], 4: [0.5090172841115191, 0.2857391744693639], 5: [0.50901728

glc__D_e_1:  -10.02301895162375
glc__D_e_1:  -24.423408992719786
glc__D_e_1:  -1.242254817497513
glc__D_e_1:  -24.786279166706393
glc__D_e_1:  -21.78084110783629
glc__D_e_1:  -1.0183112648356207
glc__D_e_1:  -0.4432705151497244
glc__D_e_1:  -14.665032478400764
glc__D_e_1:  -17.2726933780039
glc__D_e_1:  -0.3951998842303581
glc__D_e_1:  -18.91140352764605
glc__D_e_1:  -18.72717090799625
glc__D_e_1:  -0.3317739483361617
glc__D_e_1:  -16.32025209341961
glc__D_e_1:  -40.93852717469143
glc__D_e_1:  -0.36866138778999774
glc__D_e_1:  -9.929292158062115
glc__D_e_1:  -23.920981947744338
glc__D_e_1:  -35.82161499306743
glc__D_e_1:  -46.114816716113786
glc__D_e_1:  -0.3971573704958544
glc__D_e_1:  -27.75438359244396
glc__D_e_1:  -28.613377545024978
glc__D_e_1:  -0.44188574161309546
glc__D_e_1:  -13.437583433039853
glc__D_e_1:  -29.972007465035546
glc__D_e_1:  -0.28261219573711927
glc__D_e_1:  -14.605610271411345
strain_number:  0 39.233333333333334
strain_various:  0 16.13626006511078
strain_numb

glc__D_e_1:  -5.369889526289566
glc__D_e_1:  -2.2751333565291008
glc__D_e_1:  -3.6019685302595446
glc__D_e_1:  -1.9250966344991016
glc__D_e_1:  -4.719669582441728
glc__D_e_1:  -2.3140006184993807
glc__D_e_1:  -1.546664462226472
glc__D_e_1:  -0.8493669865974549
glc__D_e_1:  -1.5656791595779043
glc__D_e_1:  -1.8340770824239172
glc__D_e_1:  -2.2409553450138326
glc__D_e_1:  -2.6009574301832044
glc__D_e_1:  -1.6128127653546007
glc__D_e_1:  -0.6377299408136927
glc__D_e_1:  -1.6228181716039098
glc__D_e_1:  -0.6340329762658814
glc__D_e_1:  -1.388796562092928
glc__D_e_1:  -3.8584295441842738
glc__D_e_1:  -1.3132948592474885
glc__D_e_1:  -0.8721180184632636
glc__D_e_1:  -2.547426858405505
glc__D_e_1:  -1.5740601883106125
glc__D_e_1:  -1.1689838091043931
glc__D_e_1:  -3.629570661661111
glc__D_e_1:  -2.748860128819513
glc__D_e_1:  -0.40358536968924263
strain_number:  0 4.3
strain_various:  0 2.3402279091290805
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 